In [1]:
import collections

from nltk.corpus import wordnet as wn
import numpy as np

from src import datasets, utils
from src.evaluation import ErrorClsTypes, EvalManager

%load_ext autoreload
%autoreload 2

2023-11-03 11:33:38.715713: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-03 11:33:39.780194: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Dataset stats

In [2]:
eval_manager = EvalManager(dataset='imagenet')
df = eval_manager.init_df
# Filter the samples from the validation set that are also in the multi-label dataset:
df = df[df['has_defined_multi_labels']]
df = df.drop(columns=['has_defined_multi_labels'])

# Labels according to group
allowed_labels_per_group = eval_manager.allowed_labels_per_group
for k in allowed_labels_per_group:
    print(f'Group {k}: {len(allowed_labels_per_group[k])} classes')

# Keep only the non-problematic samples!
df = df[~df['is_problematic']]

df

2023-11-03 11:33:53.923 | INFO     | src.evaluation:__init__:78 - Initialize EvalManager
2023-11-03 11:33:54.178 | INFO     | src.utils:add_multi_labels_to_df:114 - Load multilabels from imagenet2012_multilabel
2023-11-03 11:33:54.205605: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-03 11:33:54.206216: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-03 11:33:54.207253: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU l

2023-11-03 11:34:51.290 | DEBUG    | src.utils:common_co_occurrences:278 - Appearing more than once: 1019 pairs
2023-11-03 11:34:51.355 | INFO     | src.evaluation:__init__:175 - DONE: 2038 pairs in total. (a,b) and (b,a) both counted
2023-11-03 11:34:51.357 | INFO     | src.evaluation:__init__:178 - === DoughBagel non-prototypical samples: 36 in total ===


Group all: 1000 classes
Group organism: 410 classes
Group artifact: 522 classes
Group other: 68 classes


,wnet_id,file_name,img_rel_path,target,target_desc,clip_top10_train_indices,clip_top10_train_targets,multi_label,wrong_multi_label,is_problematic,multi_desc,wrong_multi_desc,clip_top10_train_files,clip_top10_supercls
0,n01440764,ILSVRC2012_val_00000293.JPEG,n01440764/ILSVRC2012_val_00000293.JPEG,0,"tench, Tinca tinca","[1097, 59, 612, 792, 965, 852, 830, 1094, 610,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[0],"[48, 390, 391, 394, 395]",False,"[(tench, Tinca tinca)]","[(Komodo dragon, Komodo lizard, dragon lizard,...","[n01440764/n01440764_8052.JPEG, n01440764/n014...","[fish_rest, fish_rest, fish_rest, fish_rest, f..."
4,n01440764,ILSVRC2012_val_00007197.JPEG,n01440764/ILSVRC2012_val_00007197.JPEG,0,"tench, Tinca tinca","[435, 1104, 735, 921, 852, 1116, 561, 549, 173...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[0],[939],False,"[(tench, Tinca tinca)]","[(zucchini, courgette)]","[n01440764/n01440764_1735.JPEG, n01440764/n014...","[fish_rest, fish_rest, fish_rest, fish_rest, f..."
5,n01440764,ILSVRC2012_val_00009111.JPEG,n01440764/ILSVRC2012_val_00009111.JPEG,0,"tench, Tinca tinca","[560, 610, 742, 516, 529, 173, 403, 972, 1168,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[0],[],False,"[(tench, Tinca tinca)]",[],"[n01440764/n01440764_25478.JPEG, n01440764/n01...","[fish_rest, fish_rest, fish_rest, fish_rest, f..."
8,n01440764,ILSVRC2012_val_00009379.JPEG,n01440764/ILSVRC2012_val_00009379.JPEG,0,"tench, Tinca tinca","[460, 589, 835, 1044, 1142, 408, 144, 421, 699...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[0],[],False,"[(tench, Tinca tinca)]",[],"[n01440764/n01440764_19005.JPEG, n01440764/n01...","[fish_rest, fish_rest, fish_rest, fish_rest, f..."
9,n01440764,ILSVRC2012_val_00009396.JPEG,n01440764/ILSVRC2012_val_00009396.JPEG,0,"tench, Tinca tinca","[386, 33, 1122, 550, 92, 1035, 518, 243, 931, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[0],[],False,"[(tench, Tinca tinca)]",[],"[n01440764/n01440764_14771.JPEG, n01440764/n01...","[fish_rest, fish_rest, fish_rest, fish_rest, f..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49985,n15075141,ILSVRC2012_val_00038905.JPEG,n15075141/ILSVRC2012_val_00038905.JPEG,999,"toilet tissue, toilet paper, bathroom tissue","[1280247, 1280409, 1280384, 1279993, 1280639, ...","[999, 999, 999, 999, 999, 999, 999, 999, 999, ...",[999],"[471, 674]",False,"[(toilet tissue, toilet paper, bathroom tissue)]","[(cannon), (mousetrap)]","[n15075141/n15075141_20538.JPEG, n15075141/n15...","[paper_tissue, paper_tissue, paper_tissue, pap..."
49987,n15075141,ILSVRC2012_val_00039476.JPEG,n15075141/ILSVRC2012_val_00039476.JPEG,999,"toilet tissue, toilet paper, bathroom tissue","[1280154, 897812, 898599, 1280426, 1280655, 89...","[999, 700, 700, 999, 999, 700, 999, 700, 700, ...","[281, 999]","[51, 700]",False,"[(tabby, tabby cat), (toilet tissue, toilet pa...","[(triceratops), (paper towel)]","[n15075141/n15075141_1814.JPEG, n03887697/n038...","[paper_tissue, paper_tissue, paper_tissue, pap..."
49990,n15075141,ILSVRC2012_val_00041219.JPEG,n15075141/ILSVRC2012_val_00041219.JPEG,999,"toilet tissue, toilet paper, bathroom tissue","[898032, 1180622, 566334, 1181406, 1181165, 11...","[700, 922, 441, 922, 922, 909, 909, 922, 922, ...","[762, 999]","[424, 454, 466, 467, 582, 624, 700, 736, 793, ...",False,"[(restaurant, eating house, eating place, eate...","[(barbershop), (bookshop, bookstore, bookstall...","[n03887697/n03887697_18247.JPEG, n07565083/n07...","[paper_tissue, cover_presentation, beverage_po..."
49993,n15075141,ILSVRC2012_val_00042270.JPEG,n15075141/ILSVRC2012_val_00042270.JPEG,999,"toilet tissue, toilet paper, bathroom tissue","[898022, 898470, 898046, 1280417, 1279906, 128...","[700, 700, 700, 999, 999, 999, 999, 999, 700, ...",[999],"[151, 253, 700]",False,"[(toilet tissue, toilet paper, bathroom tissue)]","[(Chihuahua), (basenji), (paper towel)]","[n03887697/n03887697_18072.JPEG, n03887697/n03...","[paper_tissue, paper_tissue, paper_tissue, pap..."


In [3]:
print(len(eval_manager.common_co_occurrence_pairs_set) // 2)
for a, b in eval_manager.common_co_occurrence_pairs_set:
    if a < b:
        print(f"({a}, {b}) | ({eval_manager.get_description(a)}) ; ({eval_manager.get_description(b)})")

1019
(505, 550) | (coffeepot) ; (espresso maker)
(520, 529) | (crib, cot) ; (diaper, nappy, napkin)
(582, 998) | (grocery store, grocery, food market, market) ; (ear, spike, capitulum)
(453, 624) | (bookcase) ; (library)
(631, 896) | (lotion) ; (washbasin, handbasin, washbowl, lavabo, wash-hand basin)
(548, 632) | (entertainment center) ; (loudspeaker, speaker, speaker unit, loudspeaker system, speaker system)
(438, 617) | (beaker) ; (lab coat, laboratory coat)
(521, 651) | (Crock Pot) ; (microwave, microwave oven)
(587, 596) | (hammer) ; (hatchet)
(214, 234) | (Gordon setter) ; (Rottweiler)
(535, 671) | (disk brake, disc brake) ; (mountain bike, all-terrain bike, off-roader)
(608, 655) | (jean, blue jean, denim) ; (miniskirt, mini)
(526, 851) | (desk) ; (television, television system)
(652, 847) | (military uniform) ; (tank, army tank, armored combat vehicle, armoured combat vehicle)
(733, 920) | (pole) ; (traffic light, traffic signal, stoplight)
(728, 841) | (plastic bag) ; (sweatsh

In [4]:
for k in allowed_labels_per_group:
    # Filter samples based on multi-labels:
    # keep the sample if any of the labels belongs to the group.
    group_df = df[
        df['multi_label'].map(lambda multi_label: any(l in allowed_labels_per_group[k] for l in multi_label))
    ]
    print(f'Group: {k} ({group_df.shape[0]} samples)')
    print('% images with more than one entity: {:.2f}%'.format(
        100.0 * group_df['multi_label'].map(lambda multi_label: len(multi_label) > 1).mean()
    ))
    print('avg number of (labeled) entities per image (for all images): {:.2f}'.format(
        group_df['multi_label'].map(lambda multi_label: len(multi_label)).mean()
    ))
    print('avg number of (labeled) entities per image (for multi-object images): {:.2f}'.format(
        group_df[
            group_df['multi_label'].map(lambda multi_label: len(multi_label) > 1)
        ]['multi_label'].map(lambda multi_label: len(multi_label)).mean()
    ))
    print()

Group: all (18692 samples)
% images with more than one entity: 19.20%
avg number of (labeled) entities per image (for all images): 1.26
avg number of (labeled) entities per image (for multi-object images): 2.35

Group: organism (7624 samples)
% images with more than one entity: 10.38%
avg number of (labeled) entities per image (for all images): 1.14
avg number of (labeled) entities per image (for multi-object images): 2.32

Group: artifact (10184 samples)
% images with more than one entity: 27.88%
avg number of (labeled) entities per image (for all images): 1.38
avg number of (labeled) entities per image (for multi-object images): 2.37

Group: other (1596 samples)
% images with more than one entity: 41.98%
avg number of (labeled) entities per image (for all images): 1.55
avg number of (labeled) entities per image (for multi-object images): 2.32



In [5]:
print('Non problematic multi_label dataset size:', df.shape[0])
print('Single multi label:', df['multi_label'].map(lambda multi_label: len(multi_label) == 1).sum())
multi_df = df[df['multi_label'].map(lambda multi_label: len(multi_label) > 1)]
print(
    'Multi-label (>1 labels) with no artifact and no organism:',
    multi_df['multi_label'].map(
        lambda multi_label:
        not any(l in allowed_labels_per_group['organism'] for l in multi_label) and
        not any(l in allowed_labels_per_group['artifact'] for l in multi_label)
    ).sum()
)
print(
    'Multi-label (>1 labels) with artifact(s) and organism(s):',
    multi_df['multi_label'].map(
        lambda multi_label: 
        any(l in allowed_labels_per_group['organism'] for l in multi_label) and
        any(l in allowed_labels_per_group['artifact'] for l in multi_label)
    ).sum()
)
print(
    'Multi-label (>1 labels) with artifact(s) and no organism(s):',
    multi_df['multi_label'].map(
        lambda multi_label: 
        not any(l in allowed_labels_per_group['organism'] for l in multi_label) and
        any(l in allowed_labels_per_group['artifact'] for l in multi_label)
    ).sum()
)
print(
    'Multi-label (>1 labels) with more than 1 artifact and no organism(s):',
    multi_df['multi_label'].map(
        lambda multi_label: 
        not any(l in allowed_labels_per_group['organism'] for l in multi_label) and
        sum(l in allowed_labels_per_group['artifact'] for l in multi_label) > 1
    ).sum()
)

print(
    'Multi-label (>1 labels) with no artifact(s) and organism(s):',
    multi_df['multi_label'].map(
        lambda multi_label: 
        any(l in allowed_labels_per_group['organism'] for l in multi_label) and
        not any(l in allowed_labels_per_group['artifact'] for l in multi_label)
    ).sum()
)
print(
    'Multi-label (>1 labels) with no artifact(s) and more than 1 organism(s):',
    multi_df['multi_label'].map(
        lambda multi_label: 
        sum(l in allowed_labels_per_group['organism'] for l in multi_label) > 1 and
        not any(l in allowed_labels_per_group['artifact'] for l in multi_label)
    ).sum()
)

Non problematic multi_label dataset size: 18692
Single multi label: 15104
Multi-label (>1 labels) with no artifact and no organism: 222
Multi-label (>1 labels) with artifact(s) and organism(s): 264
Multi-label (>1 labels) with artifact(s) and no organism(s): 2575
Multi-label (>1 labels) with more than 1 artifact and no organism(s): 2292
Multi-label (>1 labels) with no artifact(s) and organism(s): 527
Multi-label (>1 labels) with no artifact(s) and more than 1 organism(s): 444


### Superclass analysis

In [6]:
supercls_to_class_ids = eval_manager.supercls_to_class_ids

for k in allowed_labels_per_group:
    print('Group: ', k)
    cnt_superclasses = 0
    cnt_singleton_superclasses = 0
    cnt_non_singleton_superclasses = 0
    all_superclass_sizes = []
    non_singleton_superclass_sizes = []
    for supercls, class_ids in supercls_to_class_ids.items():
        if not class_ids.isdisjoint(allowed_labels_per_group[k]):
            cnt_superclasses += 1
            all_superclass_sizes.append(len(class_ids))
            if len(class_ids) == 1:
                cnt_singleton_superclasses += 1
            else:
                cnt_non_singleton_superclasses += 1
                non_singleton_superclass_sizes.append(len(class_ids))
    
    print('Number of (singleton + non-singleton) superclasses:', cnt_superclasses)
    print('Number of singleton                   superclasses:', cnt_singleton_superclasses)
    print('Number of non-singleton               superclasses:', cnt_non_singleton_superclasses)
    print()
    print('All superclass stats:')
    print('min:', min(all_superclass_sizes))
    print('max:', max(all_superclass_sizes))
    print(f'avg: {np.mean(all_superclass_sizes):.1f}')
    print('median:', np.median(all_superclass_sizes))
    print()
    print('Non-singleton superclass stats:')
    print('min:', min(non_singleton_superclass_sizes))
    print('max:', max(non_singleton_superclass_sizes))
    print(f'avg: {np.mean(non_singleton_superclass_sizes):.1f}')
    print('median:', np.median(non_singleton_superclass_sizes))
    print()
    print('===')
    print()

Group:  all
Number of (singleton + non-singleton) superclasses: 235
Number of singleton                   superclasses: 74
Number of non-singleton               superclasses: 161

All superclass stats:
min: 1
max: 31
avg: 4.9
median: 3.0

Non-singleton superclass stats:
min: 2
max: 31
avg: 6.7
median: 4.0

===

Group:  organism
Number of (singleton + non-singleton) superclasses: 58
Number of singleton                   superclasses: 8
Number of non-singleton               superclasses: 50

All superclass stats:
min: 1
max: 31
avg: 8.6
median: 7.0

Non-singleton superclass stats:
min: 2
max: 31
avg: 9.8
median: 7.5

===

Group:  artifact
Number of (singleton + non-singleton) superclasses: 160
Number of singleton                   superclasses: 59
Number of non-singleton               superclasses: 101

All superclass stats:
min: 1
max: 26
avg: 3.7
median: 2.5

Non-singleton superclass stats:
min: 2
max: 26
avg: 5.3
median: 4.0

===

Group:  other
Number of (singleton + non-singleton) su

### Spurious correlation pairs analysis

In [7]:
pairs = {(min(a, b), max(a, b)) for a, b in eval_manager.common_co_occurrence_pairs_set}
organism_only = 0
artifact_only = 0
artifact_organism = 0
other_combination = 0
for a, b in pairs:
    if a in allowed_labels_per_group['organism'] and b in allowed_labels_per_group['organism']:
        organism_only += 1
    elif a in allowed_labels_per_group['artifact'] and b in allowed_labels_per_group['artifact']:
        artifact_only += 1
    elif a in allowed_labels_per_group['organism'] and b in allowed_labels_per_group['artifact']:
        artifact_organism += 1
    elif a in allowed_labels_per_group['artifact'] and b in allowed_labels_per_group['organism']:
        artifact_organism += 1
    else:
        other_combination += 1

print('organism_only =', organism_only)
print('artifact_only =', artifact_only)
print('artifact_organism =', artifact_organism)
print('other_combination =', other_combination)

organism_only = 35
artifact_only = 728
artifact_organism = 40
other_combination = 216


### Collapsed mappings

In [8]:
collapsed_mappings = utils.dough_bagel_collapsed_class_definitions()

cnt_keys_appear = 0
cnt_mappings_included = 0

for _, row in df.iterrows():
    multi_label = row.multi_label
    for k in collapsed_mappings:
        if k in multi_label:
            cnt_keys_appear += 1
            if all(p in multi_label for p in collapsed_mappings[k]):
                cnt_mappings_included += 1
            else:
                print(k, multi_label)

print('any of the keys in the collapsed mapping dictionary appear in multi labels:', cnt_keys_appear)
print('from those cases, (all of) their allowed predictions also appear in the multi labels:', cnt_mappings_included)
print(100.0 * cnt_mappings_included / cnt_keys_appear)

837 [785, 837, 161]
782 [195, 508, 782]
837 [837, 389, 758]
435 [435, 648, 794, 896]
782 [590, 782, 487]
782 [598, 851, 782, 548]
620 [526, 664, 782, 620, 508]
782 [669, 851, 782]
782 [688, 782]
664 [750, 894, 905, 851, 664, 526, 721, 619, 418]
435 [435, 794]
782 [851, 782]
837 [914, 837]
any of the keys in the collapsed mapping dictionary appear in multi labels: 1116
from those cases, (all of) their allowed predictions also appear in the multi labels: 1103
98.83512544802868


# WordNet classes

In [9]:
imagenet1k_df = utils.create_initial_samples_df()
imagenet1k_df = imagenet1k_df[['wnet_id', 'target', 'target_desc']]
imagenet1k_df.drop_duplicates(subset='wnet_id', inplace=True)
imagenet1k_df

,wnet_id,target,target_desc
0,n01440764,0,"tench, Tinca tinca"
50,n01443537,1,"goldfish, Carassius auratus"
100,n01484850,2,"great white shark, white shark, man-eater, man..."
150,n01491361,3,"tiger shark, Galeocerdo cuvieri"
200,n01494475,4,"hammerhead, hammerhead shark"
...,...,...,...
49750,n13044778,995,earthstar
49800,n13052670,996,"hen-of-the-woods, hen of the woods, Polyporus ..."
49850,n13054560,997,bolete
49900,n13133613,998,"ear, spike, capitulum"


In [10]:
def flatten(original_list):
    return [element for sublist in original_list for element in sublist]

def filter_1k_contains(args, exact_match: bool = False, print_paths: bool = False):
    cnt = 0
    filtered_set = set()
    for _, row in imagenet1k_df.iterrows():
        wnet_id = row.wnet_id
        wn_synset = wn.synset_from_pos_and_offset(wnet_id[0], int(wnet_id[1:]))
        hypernym_paths = wn_synset.hypernym_paths()
        superset_synsets = flatten(hypernym_paths)
        
        contained = False
        
        if not exact_match:
            contained = all(
                any(lemma_name in synset_lemma_name 
                    for synset in superset_synsets for synset_lemma_name in synset.lemma_names()) 
                for lemma_name in args
            )
        else:
            contained = all(
                any(lemma_name in synset.lemma_names() for synset in superset_synsets) 
                for lemma_name in args
            )
            
        if contained:
            cnt += 1
            print(f'{cnt}:', row.wnet_id, row.target_desc)
#             for synset in superset_synsets:
#                 print(synset.lemma_names())
            
            filtered_set.add((row.wnet_id, row.target_desc))
            
            if print_paths:
                for path in hypernym_paths:
                    print('*', path)
                print()
    print('Total:', cnt)
    return filtered_set

In [11]:
_ = filter_1k_contains(["crane"], exact_match=True, print_paths=True)

1: n02012849 crane
* [Synset('entity.n.01'), Synset('physical_entity.n.01'), Synset('object.n.01'), Synset('whole.n.02'), Synset('living_thing.n.01'), Synset('organism.n.01'), Synset('animal.n.01'), Synset('chordate.n.01'), Synset('vertebrate.n.01'), Synset('bird.n.01'), Synset('aquatic_bird.n.01'), Synset('wading_bird.n.01'), Synset('crane.n.05')]

2: n03126707 crane
* [Synset('entity.n.01'), Synset('physical_entity.n.01'), Synset('object.n.01'), Synset('whole.n.02'), Synset('artifact.n.01'), Synset('instrumentality.n.03'), Synset('device.n.01'), Synset('lifting_device.n.01'), Synset('crane.n.04')]

Total: 2


In [12]:
utils.print_wnet_paths('n02074367')
utils.print_wnet_paths('n02077923')

n02074367 Synset('dugong.n.01')
* [Synset('entity.n.01'), Synset('physical_entity.n.01'), Synset('object.n.01'), Synset('whole.n.02'), Synset('living_thing.n.01'), Synset('organism.n.01'), Synset('animal.n.01'), Synset('chordate.n.01'), Synset('vertebrate.n.01'), Synset('mammal.n.01'), Synset('placental.n.01'), Synset('aquatic_mammal.n.01'), Synset('sea_cow.n.01'), Synset('dugong.n.01')]
n02077923 Synset('sea_lion.n.01')
* [Synset('entity.n.01'), Synset('physical_entity.n.01'), Synset('object.n.01'), Synset('whole.n.02'), Synset('living_thing.n.01'), Synset('organism.n.01'), Synset('animal.n.01'), Synset('chordate.n.01'), Synset('vertebrate.n.01'), Synset('mammal.n.01'), Synset('placental.n.01'), Synset('aquatic_mammal.n.01'), Synset('pinniped_mammal.n.01'), Synset('seal.n.09'), Synset('eared_seal.n.01'), Synset('sea_lion.n.01')]


In [13]:
wn.synset('marlinspike.n.01').offset()

3722646